### <span style=color:blue> Loading Listings & Reviews data from postgresql into local MongoDB    </span>

In [3]:
import sys
import json
import csv
import yaml

import importlib

import math

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

from datetime import time
from datetime import date
from datetime import datetime
# with the above choices, the imported datetime.time(2023,07,01) is recognized
# from datetime import date
# from datetime import datetime

import pprint

import psycopg2
from sqlalchemy import create_engine, text as sql_text

sys.path.append('helper_functions/')
import util

In [4]:
# test that utils.py has been imported well
util.hello_world()

Hello World


<span style=color:blue>Getting PostgreSQL connection set up</span>

In [6]:
schema = "new_york_city"
port = "5432"
host = "localhost"
database = "airbnb"
password = "postgres"
connection = "postgres"

# Create the db engine 

db_eng = create_engine(f"postgresql+psycopg2://{connection}:{password}@{host}:{port}/{database}",
                       connect_args={'options': '-csearch_path={}'.format(schema)},
                       isolation_level = 'SERIALIZABLE')

print("Successfully created db engine.")

Successfully created db engine.


<span style=color:blue>Getting mongodb connection set up</span>

In [8]:
from pymongo import MongoClient

client = MongoClient()
# could have written client = MongoClient("localhost", 27017)
#                 or client = MongoClient("mongodb://localhost:27017/")

<span style=color:blue>Setting up collection "listings" in mongodb</span>

In [161]:
# I have (or will have) a database "airbnb"
db = client.airbnb

# inside the "airbnb" database, I have (or will have) a collection "listings"
listings = db.listings
print(db.list_collection_names())
# I have some other collections in my airbnb database...

['listings_test', 'listings']


### <span style=color:blue>As preparation for the next steps, I have a created table reviewm in my PostgreSQL using DBeaver, in which I dropped the comments_tsv column (because not needed) and datetime, and also renamed column "id" to "review_id" (so that it is not repeating the "id" column of the listings table).</span>

#### <span style=color:blue>Note: in my listings table the datatype of the 'last_review' column is date.  If the datatype of 'last_review' in your listings table is varchar(), then run a query in DBeaver to convert all empty string values in your 'last_review' column to the value NULL.  Then the code below should work.<span>

<span style=color:blue>In the following I focus on the query q10, which fetches a left join based on all listing_ids with prefix '10'.  This is useful for doing testing.  For your assignment you should use the left join query that includes all listings.</span>

In [162]:
import importlib
import util
# using this in case I have added stuff to util.py
importlib.reload(util)

# some other queries I was experimenting with
# q = util.build_query_full_join_listings_reviewsm()
# q = util.build_query_left_join_listings_reviewsm_null_right()

q10 = util.build_query_left_join_listings_reviewsm_10()
q = util.build_query_left_join_listings_reviewsm()

In [163]:
with db_eng.connect() as conn:
    df_ljr10 = pd.read_sql(q, con=conn)
    # df_ljr = pd.read_sql(q, con=conn)
    

In [164]:
# print(df_ljr.shape)
# should be 998,310 rows in df_ljr.  This is
#     number of records in listings whose id do not show up in reviews['listing_id'] =  11,500
#   + number of reviews                                                              = 986,810

print(df_ljr10.shape)
# you might want to check this number against what you expect based on what exploration
#    you do with DBeaver

(998310, 24)


### <span style=color:blue>The left outer join has between 0 and many records for each listing_id.  There is one record for each review about that listing.  We will now re-format this data into a list of dictionaries.  Each dictionary will have the data for one listing along with a list of all of the associated reviews. </span>

In [165]:
cols = df_ljr10.columns.tolist()
# cols = df_ljr.columns.tolist()
print(cols)

['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'number_of_reviews_ltm', 'license', 'listing_id', 'review_id', 'date', 'reviewer_id', 'reviewer_name', 'comments']


<span style=color:blue>As a first step, we build a list of dictionaries with just the listing data.  To do this we use pandas to create a new dataframe with the reviews-related columns dropped</span>

In [166]:
# to do a projection and remove duplicates
cols_of_listings = ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 
                    'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 
                    'minimum_nights', 'number_of_reviews', 'last_review', 
                    'reviews_per_month', 'calculated_host_listings_count', 
                    'availability_365', 'number_of_reviews_ltm', 'license']
cols_of_reviews = ['listing_id', 'review_id', 'date', 'reviewer_id', 
                   'reviewer_name', 'comments']

df_ljr10_new = df_ljr10.drop(cols_of_reviews, axis=1).drop_duplicates()


print(df_ljr10_new.shape)
# print(df_ljr_new.head(10))

# print(df_ljr_new.iloc[13870])

(39202, 18)


<span style=color:blue>Converting the dataframe into a list of dictionaries     </span>

In [167]:
dict_ljr10_new = df_ljr10_new.to_dict('records')
print(len(dict_ljr10_new))
pprint.pp(dict_ljr10_new[0])

39202
{'id': '897040470703986366',
 'name': 'Rental unit in Brooklyn · ★4.70 · 2 bedrooms · 1 bath',
 'host_id': 33817836,
 'host_name': 'Eric',
 'neighbourhood_group': 'Brooklyn',
 'neighbourhood': 'East Flatbush',
 'latitude': 40.64865,
 'longitude': -73.94508,
 'room_type': 'Entire home/apt',
 'price': 169.0,
 'minimum_nights': 30,
 'number_of_reviews': 20,
 'last_review': datetime.date(2024, 1, 31),
 'reviews_per_month': 2.35,
 'calculated_host_listings_count': 9,
 'availability_365': 180,
 'number_of_reviews_ltm': 20,
 'license': ''}


<span style=color:blue>Let's try loading what we have so far into MongoDB, into a temporary collection     </span>

In [168]:
# testing with a new, temporary collection
listings_test = db.listings_test

try:
    result = listings_test.insert_many(dict_ljr10_new)
    print('\nLast element of result for the last run was:')
    print(result.inserted_ids[-1:])
except Exception as e:
    print('There was an error when loading the dictionary into MongoDB:')
    print(e)



There was an error when loading the dictionary into MongoDB:
cannot encode object: datetime.date(2024, 1, 31), of type: <class 'datetime.date'>


<span style=color:blue>MongoDB does not handle dates, only datetimes.  Here is a function to convert the dates into datetimes.  (An alternative would have been to convert the dates in our table reviewsm into datetimes.)

In [169]:
# This converts date to datetime.  It also converts various kinds of
#     null values into None, which loads into MongoDB without creating errors
def convert_date_to_datetime(dt):
    if pd.isnull(dt):           # tests whether dt is None, NaN, or DaT (not a date)
        return None
    elif type(dt) == pd._libs.tslibs.nattype.NaTType:  # including this, but see below
        return None
    else:
        temp = datetime(dt.year, dt.month, dt.day)
        ts = temp.timestamp()
        new_dt = datetime.fromtimestamp(ts)
        return new_dt

# testing various cases:
# Here are four dictionaries to test with
dict1 = {'foo':1, 'date': date(2023,1,2)}
dict2 = {'goo':2, 'date': math.nan}
dict3 = {'hoo':3, 'date': None}
dict4 = {'koo':4, 'date': pd.NaT}

if pd.isnull(dict3['date']):        # pd.isnull tests whether something is 
    print("dict4['date'] tested positive as NaT")    
else:
    print("dict4['date'] did not test positive as NaT")
    

print(dict1)
dict1['date'] = convert_date_to_datetime(dict1['date'])
print(dict1)

print()
print(dict2)
dict2['date'] = convert_date_to_datetime(dict2['date'])
print(dict2)

print()
print(dict3)
dict3['date'] = convert_date_to_datetime(dict3['date'])
print(dict3)

print()
print(dict4)
dict4['date'] = convert_date_to_datetime(dict4['date'])
print(dict4)

dict4['date'] tested positive as NaT
{'foo': 1, 'date': datetime.date(2023, 1, 2)}
{'foo': 1, 'date': datetime.datetime(2023, 1, 2, 0, 0)}

{'goo': 2, 'date': nan}
{'goo': 2, 'date': None}

{'hoo': 3, 'date': None}
{'hoo': 3, 'date': None}

{'koo': 4, 'date': NaT}
{'koo': 4, 'date': None}


<span style=color:blue>Use pandas to replace the dates in the "last_review" column with datetimes</span>

In [170]:
# trying to replace all dates by datetimes (or None)

# df_ljr10_new['last_review'] = df_ljr10_new['last_review'].apply(convert_date_to_datetime)

# could also have written
df_ljr10_new['last_review'] = df_ljr10_new['last_review'].apply(lambda x: convert_date_to_datetime(x))

In [171]:
print(df_ljr10_new.head())

                   id                                               name  \
0  897040470703986366  Rental unit in Brooklyn · ★4.70 · 2 bedrooms ·...   
2              513688  Rental unit in Brooklyn · ★4.89 · 3 bedrooms ·...   
3              769786  Rental unit in Brooklyn · ★4.60 · 1 bedroom · ...   
4  882802659322534873  Rental unit in Brooklyn · ★4.83 · 1 bedroom · ...   
5              112435  Home in Brooklyn · ★4.63 · 1 bedroom · 1 bed ·...   

    host_id     host_name neighbourhood_group  neighbourhood  latitude  \
0  33817836          Eric            Brooklyn  East Flatbush  40.64865   
2   2530670       Tiffany            Brooklyn    Boerum Hill  40.68607   
3   4050489  Pete & Halle            Brooklyn   Williamsburg  40.71302   
4  92735223         Milos            Brooklyn   Williamsburg  40.71595   
5    181376         Carol            Brooklyn    Fort Greene  40.69132   

   longitude        room_type  price  minimum_nights  number_of_reviews  \
0  -73.94508  Entire ho

In [172]:
# As you can see in the result from the last cell,
#   somehow the NaT's are still there, in spite of the special case included in
#   the function convert_time_to_timestamp()
#   BTW, curiously, on very small dataframes the convert_time_to_timestamp() does convert NaT to None

# Happily, all of the actual dates have converted into datetimes, as illustrated by the following:
#    Using "iloc" because the index values in df_ljr10_new are not consecutive
print(type(df_ljr10_new.iloc[0, 12]))  # 12 is position of 'last_review'
print(df_ljr10_new.iloc[0,12])
print(type(df_ljr10_new.iloc[1, 12]))  
print(df_ljr10_new.iloc[1,12])
print(type(df_ljr10_new.iloc[2, 12]))  
print(df_ljr10_new.iloc[2,12])
print(type(df_ljr10_new.iloc[3, 12]))  
print(df_ljr10_new.iloc[3,12])

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2024-01-31 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2023-10-20 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016-10-02 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2023-10-13 00:00:00


In [173]:
# recomputing dict_ljr10_new
dict_ljr10_new = df_ljr10_new.to_dict('records')
print(len(dict_ljr10_new))
pprint.pp(dict_ljr10_new[0:2])

39202
[{'id': '897040470703986366',
  'name': 'Rental unit in Brooklyn · ★4.70 · 2 bedrooms · 1 bath',
  'host_id': 33817836,
  'host_name': 'Eric',
  'neighbourhood_group': 'Brooklyn',
  'neighbourhood': 'East Flatbush',
  'latitude': 40.64865,
  'longitude': -73.94508,
  'room_type': 'Entire home/apt',
  'price': 169.0,
  'minimum_nights': 30,
  'number_of_reviews': 20,
  'last_review': Timestamp('2024-01-31 00:00:00'),
  'reviews_per_month': 2.35,
  'calculated_host_listings_count': 9,
  'availability_365': 180,
  'number_of_reviews_ltm': 20,
  'license': ''},
 {'id': '513688',
  'name': 'Rental unit in Brooklyn · ★4.89 · 3 bedrooms · 4 beds · 2 baths',
  'host_id': 2530670,
  'host_name': 'Tiffany',
  'neighbourhood_group': 'Brooklyn',
  'neighbourhood': 'Boerum Hill',
  'latitude': 40.68607,
  'longitude': -73.98074,
  'room_type': 'Entire home/apt',
  'price': nan,
  'minimum_nights': 30,
  'number_of_reviews': 249,
  'last_review': Timestamp('2023-10-20 00:00:00'),
  'reviews_pe

In [174]:
# However, the load into MongoDB still fails, because of the NaT values
#    As noted above, the convert_time_to_timestamp did not convert the NaT values
try:
    result = listings_test.insert_many(dict_ljr10_new)
    print('\nLast element of result for the last run was:')
    print(result.inserted_ids[-1:])
except Exception as e:
    print('\nThere was an error when loading the dictionary into MongoDB:')
    print(e)


There was an error when loading the dictionary into MongoDB:
NaTType does not support utcoffset


<span style=color:blue>OK, so let's convert the NaT's in the dictionary rather than in pandas  </span>

In [175]:
for doc in dict_ljr10_new:
    if pd.isnull(doc['last_review']): 
        doc['last_review'] = None

pprint.pp(dict_ljr10_new[0:10])

[{'id': '897040470703986366',
  'name': 'Rental unit in Brooklyn · ★4.70 · 2 bedrooms · 1 bath',
  'host_id': 33817836,
  'host_name': 'Eric',
  'neighbourhood_group': 'Brooklyn',
  'neighbourhood': 'East Flatbush',
  'latitude': 40.64865,
  'longitude': -73.94508,
  'room_type': 'Entire home/apt',
  'price': 169.0,
  'minimum_nights': 30,
  'number_of_reviews': 20,
  'last_review': Timestamp('2024-01-31 00:00:00'),
  'reviews_per_month': 2.35,
  'calculated_host_listings_count': 9,
  'availability_365': 180,
  'number_of_reviews_ltm': 20,
  'license': '',
  '_id': ObjectId('665dfd493f5206e632b174a9')},
 {'id': '513688',
  'name': 'Rental unit in Brooklyn · ★4.89 · 3 bedrooms · 4 beds · 2 baths',
  'host_id': 2530670,
  'host_name': 'Tiffany',
  'neighbourhood_group': 'Brooklyn',
  'neighbourhood': 'Boerum Hill',
  'latitude': 40.68607,
  'longitude': -73.98074,
  'room_type': 'Entire home/apt',
  'price': nan,
  'minimum_nights': 30,
  'number_of_reviews': 249,
  'last_review': Timest

<span style=color:blue>Now trying the load again    </span>

In [176]:
try:
    result = listings_test.insert_many(dict_ljr10_new)
    print('\nLast element of result for the last run was:')
    print(result.inserted_ids[-1:])
except Exception as e:
    print('\nThere was an error when loading the dictionary into MongoDB:')
    print(e)


Last element of result for the last run was:
[ObjectId('665dfd493f5206e632b20dca')]


<span style=color:blue>Now we add, for each listing, a list of all reviews for that listing     </span>

In [178]:
i = 0

# We will keep track of the time to do each 1000 listings
time1 = datetime.now()


for d in dict_ljr10_new:
    i += 1

    # building a df with just reviews info, and corresponding to the listing we are focusing on
    df_reviews_one_listing = df_ljr10.loc[df_ljr10['id'] == d['id']].drop(cols_of_listings, axis=1)

    # Note: This does not run super quickly.  As an alternative I tried pulling this 
    #    data with a query against PostgreSQL, but it was even slower

    # there are no null values in the 'date' column of reviews, so we can do the
    #    date to datetime conversion using pandas
    df_reviews_one_listing['date'] = df_reviews_one_listing['date'].apply(lambda x: convert_date_to_datetime(x))

    dicts_reviews_one_listing = df_reviews_one_listing.to_dict('records')

    # Need special handling for the case of no reviews 
    if len(dicts_reviews_one_listing) == 1 and dicts_reviews_one_listing[0]['review_id'] is None:
        d['reviews'] = {}
    else:
        d['reviews'] = dicts_reviews_one_listing

    if i % 1000 == 0:
        time2 = datetime.now()
        time_taken = util.time_diff(time1,time2)
        print('Have now completed step number:', str(i), 'and it took', str(time_taken), 'seconds' )
        time1 = datetime.now()

    # given the time it takes to do 1000 listings, how long will it take to do all of the listings?

print()
print(len(dict_ljr10_new))
print()
pprint.pp(dict_ljr10_new[-10:])

Have now completed step number: 1000 and it took 95.909551 seconds
Have now completed step number: 2000 and it took 90.954787 seconds
Have now completed step number: 3000 and it took 93.481388 seconds
Have now completed step number: 4000 and it took 87.021799 seconds
Have now completed step number: 5000 and it took 90.997289 seconds
Have now completed step number: 6000 and it took 89.153266 seconds
Have now completed step number: 7000 and it took 108.153255 seconds
Have now completed step number: 8000 and it took 88.068773 seconds
Have now completed step number: 9000 and it took 154.371538 seconds
Have now completed step number: 10000 and it took 115.892938 seconds
Have now completed step number: 11000 and it took 104.153967 seconds
Have now completed step number: 12000 and it took 108.991405 seconds
Have now completed step number: 13000 and it took 100.057489 seconds
Have now completed step number: 14000 and it took 92.921952 seconds
Have now completed step number: 15000 and it took 9

<span style=color:blue>Sanity check, that we did not lose any listings </span>

In [196]:
print(len(dict_ljr10_new))

39202


<span style=color:blue>Now loading dict_ljr10_new into mongodb.   </span>

<span style=color:blue>The loading is done 100 documents at a time, with a last small lot </span>

In [212]:
print(len(dict_ljr10_new) % 100)
print(len(dict_ljr10_new))

2
39202


In [182]:
# CAUTION: the first step here erases db.listing
#    I have kept this here during testing
#db.listings.drop()


listings = db.listings

time0 = datetime.now()
time1 = datetime.now()

for i in range(0,33):
# for i in range(0,10):
    result = listings.insert_many(dict_ljr10_new[100*i:100*(i+1)])

    time2 = datetime.now()
    time_taken = util.time_diff(time1,time2)
    print('Have now completed step number:', str(i), 'and it took', str(time_taken), 'seconds' )
    time1 = datetime.now()
    
time3 = datetime.now()


print('\nThe last ObjectID in the collection is:')
print(result.inserted_ids[-1:])

# print('\nThe time to do the load of 39K documents into local mongodb, with a total of about 300MB was:')
print('\nThe time for this run was:')
print(util.time_diff(time0,time3))



# this is for the last 13 records in dict_ljr10_new, but built for arbitrary number of records
result = listings.insert_many(dict_ljr10_new[3300:])


print('\nThe total number of documents in the collection db.listings is now:')
print(listings.count_documents({}))

print('\nLast few ObjectIds of result for the last run was:')
print(result.inserted_ids[-5:])

print('\nThe last few documents of result for the last run was:')
# Curiously, the next line fetches only the object ids, not the full documents
# out = listings.find({'_id' : {'$in' : result.inserted_ids[-5:]} } )
# pprint.pp(out)
outdocs = []
for o in result.inserted_ids[-5:]:
    outdocs.append(listings.find_one({ '_id': o}))
pprint.pp(outdocs)

Have now completed step number: 0 and it took 0.655299 seconds
Have now completed step number: 1 and it took 0.398229 seconds
Have now completed step number: 2 and it took 0.375838 seconds
Have now completed step number: 3 and it took 0.259969 seconds
Have now completed step number: 4 and it took 0.224444 seconds
Have now completed step number: 5 and it took 0.182194 seconds
Have now completed step number: 6 and it took 0.210087 seconds
Have now completed step number: 7 and it took 0.23307 seconds
Have now completed step number: 8 and it took 0.149034 seconds
Have now completed step number: 9 and it took 0.119776 seconds
Have now completed step number: 10 and it took 0.132876 seconds
Have now completed step number: 11 and it took 0.103347 seconds
Have now completed step number: 12 and it took 0.083753 seconds
Have now completed step number: 13 and it took 0.111682 seconds
Have now completed step number: 14 and it took 0.088752 seconds
Have now completed step number: 15 and it took 0.12

<span style=color:blue>Here is a query testing against the 'last_review' values    </span>

In [197]:
cursor = listings.find( { 'last_review' : { '$lte' : datetime(2024,1,1,0,0,0,0)}})
l = list(cursor)
print(len(l))
# pprint.pp(l)

24534


<h1>My Code (DO NOT RUN CODE ABOVE WILL MESS UP DATA)</h1>

In [9]:
from pymongo import MongoClient

client = MongoClient()
db = client.airbnb
listings = db.listings

In [10]:

query_1 = listings.count_documents({
    "last_review": {
        "$gte": datetime(2021, 2, 1),
        "$lte": datetime(2023, 3, 15)
    }
})
print("Query 1:", query_1)


Query 1: 3685


In [11]:
query_2 = [
    {
        "$match": {
            "reviews": {"$exists": True, "$type": "array"}
        }
    },
    {
        "$project": {
            "numberOfReviews": {"$size": "$reviews"}
        }
    },
    {
        "$match": {
            "numberOfReviews": {"$gte": 50}
        }
    },
    {
        "$count": "50Reviews"
    }
]

# Execute the aggregation pipeline
query_2 = list(listings.aggregate(query_2))
query_2 = query_2[0]['50Reviews']
print("Query 2:", query_2)

Query 2: 5658


In [12]:
query_3 = [
    {
        "$match": {
            "reviews": {
                "$elemMatch": {
                    "comments": {
                        "$regex": "awesome|amazing",
                    }
                }
            }
        }
    },
    {
        "$count": "listingsWithSpecificWords"
    }
]

query_3 = list(listings.aggregate(query_3))
query_3 = query_3[0]['listingsWithSpecificWords']
print("Query 3:", query_3)

Query 3: 15890


In [13]:
query_4 = [
    {
        "$match": {
            "reviews": {
                "$elemMatch": {
                    "comments": {
                        "$regex": "awesome|amazing",
                        "$options": "i"
                    }
                }
            }
        }
    },
    {
        "$count": "listingsWithSpecificWords"
    }
]

query_4 = list(listings.aggregate(query_4))
query_4 = query_4[0]['listingsWithSpecificWords']
print("Query 4:", query_4)

Query 4: 17121


In [219]:
df = pd.DataFrame({
    "Query Number": [1, 2, 3, 4],
    "Count": [query_1, query_2, query_3, query_4]
})

# Write DataFrame to a CSV file
df.to_csv('prob_set_5.csv', index=False)

print("CSV file has been created.")

CSV file has been created.
